<a href="https://colab.research.google.com/github/chetools/CHE4061_Fall2024/blob/main/BubbleDewCalculations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.6 MB/s eta 0:00:00


In [3]:
from importnb import Notebook
with Notebook():
    from che5 import Props

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

In [12]:
p=Props(["Benzene","Toluene","pXylene"])

In [13]:
x=np.array([0.3, 0.6,0.1])

np.sum(x*p.Pvap(373.15))

Array(101590.50432391, dtype=float64)

In [8]:
x[0]

0.4

In [9]:
x[1]

0.6